In this notebook I will demonstrate by genome assembly algorithm to assemble the genome of *Carsonella ruddii* that has an amazingly small genome of only around 160,000 basepairs. Because genome assembly is an incredibly difficult task that even state-of-the-art algorithms cannot solve with the current read lengths generated by sequencing machines, I will simplify the assembly to only contain error-free paired-end reads.

The approach to assemble reads is to generate a de Bruijn graph from the reads. In a de Bruijn graph, reads are broken into short kmers which represent the edges of a graph and the nodes are the (k-1)-mers that connect the kmers. From this, an Eulerian path needs to be walked that visits each edge exactly once to reconstruct the genome.

![example](de_bruijn_graph.png)

In this example I will use paired reads, which are short reads that are separated by a known distance. This is used by many sequencing protocols to make assembly of shorter reads easier. I will start with the complete genome of *Carsonella ruddii* and first chop this off into paired reads.

In [8]:
# get all k-mers of a sequence
def get_kmers(input, k):
    return [input[i:i+k] for i in range(len(input)-k+1)]

# read in the genome
with open('carsonella_ruddii_genome.txt') as f:
    genome = f.read()

# I will use reads of length 150 (k) and distance between reads of 1,000 bp (d)
k = 150
d = 1000

# get all kmers
reads = get_kmers(genome, k+d+k)

# make into paired reads with a distance of 1000 between reads and separate with a "|"
paired_reads = [(read[0:k] + "|" + read[k+d+k-k:k+d+k]) for read in reads]

Now that we have the paired reads of the genome of *Carsonella ruddii*, we can make the de Bruijn graph from these.

In [9]:
# get the prefix for a read pair
def paired_prefix(pair):
    return (pair[0][:-1], pair[1][:-1])

# get the suffix for a read pair
def paired_suffix(pair):
    return (pair[0][1:], pair[1][1:])

# make the de Bruijn graph from the paired reads
def de_bruijn_graph_from_read_pairs(read_pairs):
    # make an empty graph
    graph = {}
    
    # split the read_pairs at '|'
    for pattern in read_pairs:
        split = pattern.split("|")
        
        suffix = paired_suffix(split)
        prefix = paired_prefix(split)
        
        # add prefix:suffix pairs
        if prefix in graph.keys():
            graph[prefix].append(suffix)
        else:
            graph[prefix] = [suffix]
    
    
    return graph

graph = de_bruijn_graph_from_read_pairs(paired_reads)

From the de Bruijn graph, we want to get the Eulerian path that passes each edge exactly once. In a perfectly closed graph, we can keep randomly walking across the graph while keeping track of unexplored nodes until no unexplored nodes are left. However, even in this genome with perfect coverage, the graph is not closed because the DNA sequence is not circular. Therefore, we can use the Eulerian_path algorithm to connect the start- and end-node with an edge and then find the Eulerian cycle that spells out the complete genome. This will take some minutes.

In [10]:
import random

# walk a random path through a graph but check for unbalanced nodes
def random_walk(graph, start):
    # make a list of all nodes
    node_list = [key for key in graph.keys()]
    
    # start the cycle at the starting node
    cycle = [start]

    # make a random cycle
    while True:
        # check if this is an unbalanced node by seeing if the node is in the node list
        if cycle[-1] not in node_list:
            break
        # make a random choice for edges at each node
        choice = random.choice(graph[cycle[-1]])
        graph[cycle[-1]].remove(choice)
        cycle.append(choice)
        # when the cycle is complete break
        if cycle[-1] == start:
            break
    
    return cycle

# Get the Eulerian cycle of an Eulerian directed graph
def eulerian_cycle(graph, start):
    # make a list of all nodes
    node_list = [key for key in graph.keys()]

    # choose a random starting node if no start node is given
    if not isinstance(start, int) or isinstance(start, str):
        start = random.choice(list(graph.keys()))

    # walk a random cycle starting from the start node
    cycle = random_walk(graph, start)

    # keep track of the unexplored nodes
    unexplored = [i for i in node_list if len(graph[i]) != 0]

    # continue walking random cycles until no unexplored nodes are left
    while len(unexplored) != 0:
        # make a new graph with only the unexplored nodes
        new_graph = {key: graph[key] for key in unexplored}
        # choose a new random starting node from the list of unexplored nodes
        new_start = random.choice([ele for ele in cycle if ele in unexplored])
        # walk a new random cycle
        new_cycle = random_walk(new_graph, new_start)
        # first redo the previous cycle starting at the new start_point
        index = cycle.index(new_start)
        if index != 0:
            old_cycle = cycle[index::] + cycle[1:index]
        else:
            old_cycle = cycle[:-1]
        # the new cycle is the combination of the old cycle and the new cycle
        cycle = old_cycle + new_cycle
        # update the unexplored nodes
        unexplored = [i for i in node_list if len(graph[i]) != 0]

    # return the final Eulerian path
    return cycle

# Get the Eulerian path of an Eulerian directed graph
def eulerian_path(graph):
    # make a list of all nodes
    node_list = [key for key in graph.keys()]
    
    # make a list of all edges
    edge_list = []
    for i in graph.values():
        for value in i:
            edge_list.append(value)

    # add missing node if there is one
    if len(edge_list) != len(node_list)+1:
        difference = set(edge_list) - set(node_list)
        graph.update({list(difference)[0]: []})

    # find unbalanced nodes
    unbalanced_nodes = [key for key, value in graph.items() if len(value) != edge_list.count(key)]

    # determing which is the first and which is the last node
    # outgoing number
    outgoing_nodes = len(graph[unbalanced_nodes[0]])
    # incoming number
    incoming = []
    for ele in graph.values():
        for value in ele:
            incoming.append(value)
    # outgoing number
    incoming_nodes = incoming.count(unbalanced_nodes[0])

    if incoming_nodes < outgoing_nodes:
        start_node = unbalanced_nodes[0]
        end_node = unbalanced_nodes[1]
    else:
        start_node = unbalanced_nodes[1]
        end_node = unbalanced_nodes[0]

    # append the edge between end and start node or create if it does not yet exist
    if end_node in graph:
        graph[end_node].append(start_node)
    else:
        graph.update({end_node: [start_node]})
    
    # run eulerian_cycle
    output = eulerian_cycle(graph, start = start_node)
    
    #return start_node[0]
    # the path has to be reordered to start at the start_node and end at the end_node but
    # only if the correct starting node is not already at the start!
    if output[0] != start_node:
        start_indexes = [index for index, ele in enumerate(output) if ele == start_node]
        end_index = output.index(end_node)
        start_position = [ele for ele in start_indexes if ele == end_index+1][0]
        # return the reordered eulerian path
        return output[start_position::]+output[1:start_position]
    else:
        return output[:-1]

# walk to Eulerian path through the de Bruijn graph
path = eulerian_path(graph)

Now that we have the Eulerian path, the reads can be overlapped to generate the final assembly.

In [11]:
# convert the tuple to a list readable by string_gapped_patterns
input = []
for ele in path:
    input.append("|".join(list(ele)))

def string_spelled_by_patterns(patterns):
    str = patterns[0]
    for i in range(1, len(patterns)):
        str += patterns[i][-1]
    return str

# this function overlaps strings with gapped patterns
def string_gapped_patterns(patterns, k, d):
    # first extract the first and last kmers between the gap
    first_patterns = []
    last_patterns = []
    for pattern in patterns:
        split = pattern.split("|")
        first_patterns.append(split[0])
        last_patterns.append(split[1])

    # reconstruct the first patterns
    prefix_string = string_spelled_by_patterns(first_patterns)
    # reconstruct the second patterns
    suffix_string = string_spelled_by_patterns(last_patterns)

    # check if they match and if so return the complete string
    for i in range(k+d+1, len(prefix_string)):
        if prefix_string[i] != suffix_string[i-k-d]:
            return "there is no string spelled by the gapped patterns"
    return prefix_string + suffix_string[len(suffix_string)-k-d:]

assembly = string_gapped_patterns(input, k, d)

In [12]:
print(f"The length of the genome of Carsonella ruddii is {len(genome)} bp")
print(f"The length of the assembly is {len(assembly)} bp")

print(f"Is the assembly the same as the genome? {assembly == genome}")

The length of the genome of Carsonella ruddii is 166163 bp
The length of the assembly is 166163 bp
Is the assembly the same as the genome? True


Although this example is quite a simple one with full coverage of error-free reads, it has shown me that genome assembly from short reads is a very difficult task. When sequencing errors are introduced to the reads, finding a Eulerian path suddenly becomes extremely non-trivial. Therefore, scaffolding to a reference genome is needed in order to try to correctly order contigs.

This project did teach me a lot about the mechanism of genome assembly and it's pittfalls, which allows me to better utilize assembly software in the future. It also further taught me valuable skills on writing algorithms in Python.